In [1]:
import numpy as np 
import pandas as pd 
from pandas import *
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import *
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report 


In [2]:
df = pd.read_csv("../input/creditcardfraud/creditcard.csv", delimiter=",")

df.head()

In [3]:
# The data is shown

sns.countplot("Class", data=df)
plt.title('Not Fraud=0 vs Fraud=1')


In [4]:
# realize that time and amount need to be scaled
# source used: https://www.kaggle.com/code/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets

std_scaler = StandardScaler()
rob_scaler = RobustScaler()

df["scaled_amt"] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df["scaled_time"] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))
df.drop(['Time','Amount'], axis=1, inplace=True)

scaled_amt = df['scaled_amt']
scaled_time = df['scaled_time']

df.drop(['scaled_amt', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amt', scaled_amt)
df.insert(1, 'scaled_time', scaled_time)
df.head()


In [5]:
normal_df = df
normal_df

In [6]:
# evening out the data

number_of_honest = len(df[df.Class == 0])
number_of_frauds = len(df[df.Class == 1])
# therefore need to create train-test numbering 492 fraud and 492 non-fraud, where num_frauds becomes the bottleneck
                    
# select randomly from honest (indices)                    
honest_index = df[df.Class == 0].index

random_honest_indices = np.random.choice(honest_index, number_of_frauds, replace=False)
random_honest_indices = np.array(random_honest_indices)

# indicize frauds
fraud_indices = np.array(df[df.Class == 1].index)

# new dataset
new_indices = np.concatenate([random_honest_indices, fraud_indices])
# print(new_indices)

# get new data and store into df1
even_data = df.iloc[new_indices,:]
# print(new_data)

df1 = pandas.DataFrame(even_data)


In [7]:
# new data where half are frauds, and half are honest
df1

In [8]:
# Collect X_new, y_new

X_new = df1.iloc[:,:-1].values
y_new = np.array(df1['Class'])
# print(X_new)

# do train_test_split on data
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.33, random_state=42)


# normal df with ALL THE DATA
X_org = normal_df.iloc[:,:-1].values
y_org = np.array(normal_df['Class'])
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_org, y_org, test_size=0.33, random_state=42)

# y_train.shape
# print(len(X_train)+len(X_test))
# print(X_train)


In [16]:

none = []
l2 = []

for c in [0.01, 0.1, 1, 10, 100, 1000]:
    log_reg = linear_model.LogisticRegression(C=c, random_state = 0, penalty = "l1", solver='liblinear')
    l2_reg = linear_model.LogisticRegression(C=c, random_state = 0, penalty="none")

    log_reg.fit(X_train, y_train)
    l2_reg.fit(X_train, y_train)

    y_pred = log_reg.predict(X_test)
    y_pred_l2 = l2_reg.predict(X_test)

    none.append(log_reg.score(X_test, y_test))
#     none.append(metrics.precision_score(y_test, y_pred))
#     none.append(metrics.recall_score(y_test, y_pred))
#     none.append(metrics.f1_score(y_test, y_pred))

    l2.append(log_reg.score(X_test, y_test))
#     l2.append(metrics.precision_score(y_test, y_pred_l2))
#     l2.append(metrics.recall_score(y_test, y_pred_l2))
#     l2.append(metrics.f1_score(y_test, y_pred_l2))

none_org = []
l2_org = []
for c in [0.01, 0.1, 1, 10, 100, 1000]:
    log_reg_org = linear_model.LogisticRegression(C=c, random_state = 0, penalty = "l1", solver='liblinear')
    l2_reg_org = linear_model.LogisticRegression(C=c, random_state = 0, penalty="none")

    log_reg_org.fit(X_train1, y_train1)
    l2_reg_org.fit(X_train1, y_train1)

    y_pred_org = log_reg_org.predict(X_test1)
    y_pred_l2_org = l2_reg_org.predict(X_test1)

    none_org.append(log_reg_org.score(X_test1, y_test1))
#     none.append(metrics.precision_score(y_test, y_pred))
#     none.append(metrics.recall_score(y_test, y_pred))
#     none.append(metrics.f1_score(y_test, y_pred))

    l2_org.append(log_reg_org.score(X_test1, y_test1))
#     l2.append(metrics.precision_score(y_test, y_pred_l2))
#     l2.append(metrics.recall_score(y_test, y_pred_l2))
#     l2.append(metrics.f1_score(y_test, y_pred_l2))



In [17]:
c = [0.01, 0.1, 1, 10, 100, 1000]
# none on new dataset
#actually none is l1 just named it wrong
print(c)
print(none)
print(l2)

matrix = metrics.confusion_matrix(y_test, y_pred)
sns.heatmap(matrix, annot=True, fmt="d")
plt.title("Confusion matrix for processed dataset")
plt.ylabel("True Class")
plt.xlabel("Predicted Class")
plt.show()



In [19]:
# one original dataset
print(none_org)
print("original accuracy: ",l2_org)

matrix = metrics.confusion_matrix(y_test1, y_pred_l2_org)
sns.heatmap(matrix, annot=True, fmt="d")
plt.title("Confusion matrix tested against original dataset")
plt.ylabel("True Class")
plt.xlabel("Predicted Class")
plt.show()

In [12]:

# # Logistic Regression
# import statistics
# # testing for c_param
# c_param = [0.01, 0.1, 1, 10, 100, 1000]

# def k_fold(X_train, y_train, c_param, title):
    
#     #shuffled here so must create a y_pred
#     kf = KFold(n_splits=5, shuffle=True)
    
#     # show to table
#     table_results = pd.DataFrame(index = range(len(c_param),2), columns = ['c_param','Mean recall score'])
#     table_results['C_parameter'] = c_param
    
#     #begin the training
#     recall_list_l2 = []
#     recall_list = []
#     for i in c_param:
#         print()
#         print('----------------')
#         print(f"for c_param = {i}")
#         print('----------------')
            
#         itr=1
#         recall_temp_l2=[]
#         recall_temp=[]
#         for train_index, test_index in kf.split(X_train):
            
#             #l2
#             lr_l2 = LogisticRegression(C=i, penalty="l2")
#             #nopenalty
#             lr = LogisticRegression(C=i)
            
#             lr.fit(X_train,y_train)
#             lr_l2.fit(X_train,y_train)
            
#             #retreive class labels for samples in X
#             y_pred_l2 = lr_l2.predict(X_train)
#             y_pred = lr.predict(X_train)

#             #l2
#             recall_acc_l2 = recall_score(y_train, y_pred_l2)
#             recall_list_l2.append(recall_acc_l2)
#             recall_temp_l2.append(recall_acc_l2)
            
#             #l1
#             recall_acc = recall_score(y_train, y_pred)
#             recall_list.append(recall_acc)
#             recall_temp.append(recall_acc)
            
#             print(f"L2: iteration={itr}   recall acc={recall_acc_l2}")
#             print(f"l1: iteration={itr}   recall acc={recall_acc}")
#             itr+=1
# #             retreive mean recall score
#         mean_recall = statistics.mean(recall_temp)
#         mean_recall_l2 = statistics.mean(recall_temp_l2)
    
#         print("mean recall l1=", mean_recall)
#         print("mean recall l2=", mean_recall_l2)
             

#     default_x_ticks = range(len(c_param))
#     plt.xticks(default_x_ticks, c_param)
    
#     plt.plot(default_x_ticks, recall_list[0:6])
#     plt.plot(default_x_ticks, recall_list[6:12])
#     plt.plot(default_x_ticks, recall_list[12:18])
#     plt.plot(default_x_ticks, recall_list[18:24])
#     plt.plot(default_x_ticks, recall_list[24:30])
    
#     plt.plot(default_x_ticks, recall_list_l2[0:6])
#     plt.plot(default_x_ticks, recall_list_l2[6:12])
#     plt.plot(default_x_ticks, recall_list_l2[12:18])
#     plt.plot(default_x_ticks, recall_list_l2[18:24])
#     plt.plot(default_x_ticks, recall_list_l2[24:30])
    
# #         
#     plt.xlabel('C values') 
#     plt.ylabel('Recall accuracies') 
#     plt.title(title) 
#     plt.ylim((0.8,1.01))
#     plt.legend(c_param)
#     plt.show()
        
#     return recall_list, recall_list_l2


In [13]:
# recall_test, recall_test_l2 = k_fold(X_test, y_test, c_param, 'TEST SET Recall accuracies vs C values')
# recall_train, recall_train_l2 = k_fold(X_train, y_train, c_param, 'TRAININING Recall accuracies vs C values')

In [20]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

def svm_(X_train, y_train, X_test, y_test, c):
    score_lin = []
    score_poly = []
    score_rbf = []
    
    for c_val in c:
    
    # polyial based
        poly_classifier = svm.SVC(C=c_val, kernel='poly', random_state = 0)
    
    # linear based
        lin_classifier = svm.SVC(C=c_val, kernel='linear', random_state=0)
        
        # linear based
        rbf_classifier = svm.SVC(C=c_val, kernel='rbf', random_state=0)
        
        rbf_classifier.fit(X_train, y_train)
        lin_classifier.fit(X_train, y_train)
        poly_classifier.fit(X_train, y_train)
    
    # prediction
        rbf_pred = rbf_classifier.predict(X_test)
        lin_pred = lin_classifier.predict(X_test)
        poly_pred = poly_classifier.predict(X_test)
    
    # score
        rbf_score = metrics.accuracy_score(rbf_pred, y_test)
        lin_score = metrics.accuracy_score(lin_pred, y_test)
        poly_score = metrics.accuracy_score(poly_pred, y_test)
        
        score_rbf.append(rbf_score)
        score_lin.append(lin_score)
        score_poly.append(poly_score)
#         print('Accuracy score for lin: {} is {}'.format(c_val, lin_score))
#         print('Accuracy score for poly: {} is {}'.format(c_val, poly_score))

    prfs_lin = precision_recall_fscore_support(y_test, lin_pred, average='macro')
    prfs_poly = precision_recall_fscore_support(y_test, poly_pred, average='macro')
    prfs_rbf = precision_recall_fscore_support(y_test, rbf_pred, average='macro')
    
    cm_rbf = confusion_matrix(y_test, rbf_pred)
    cm_linear = confusion_matrix(y_test, lin_pred)
    cm_polyial = confusion_matrix(y_test, poly_pred)
    
    print(cm_rbf)
    print(cm_linear)
    print(cm_polyial)

    return cm_linear, cm_polyial, cm_rbf, prfs_lin, prfs_poly, prfs_rbf, score_lin, score_poly, score_rbf


## TESTING ON PREPROCESSED DATA


In [47]:
c_param = [0.01, 0.1, 1, 10, 100, 1000]
test_linear_conf, test_polyial_conf, test_rbf_conf, prfs_lin, prfs_poly, prfs_rbf, s1, s2,s3 = svm_(X_train, y_train, X_test, y_test,c_param)

In [52]:
plt.plot(c_param, s1, label = 'Linear Regression')
plt.plot(c_param, s2, label = 'Polynomial Regression')
plt.plot(c_param, s3, label = 'RBF Regression')
plt.xscale('log')
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title('Accuracy vs C balanced')
plt.legend()
plt.show()

# C = 0.1 is the optimal value

In [49]:
#     linear kernel

vals = test_linear_conf

plt.imshow(test_linear_conf, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted')
plt.title('Linear Kernel Confusion')

s = [['TP', 'FP'], ['FN', 'TN']]

for i in range(2):
    for j in range(2):
        plt.text(j, i, str(s[i][j]) + ' = ' + str(vals[i][j]))


print(test_linear_conf)
print("precision", prfs_lin[0])
print("recall", prfs_lin[1])
print("f-score", prfs_lin[2])

In [50]:
# polyial kernel

vals = test_polyial_conf

plt.imshow(test_polyial_conf, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted')
plt.title('SVM Poly Confusion')

s = [['TP', 'FP'], ['FN', 'TN']]

for i in range(2):
    for j in range(2):
        plt.text(j, i, str(s[i][j]) + ' = ' + str(vals[i][j]))

print(test_linear_conf)
print("precision", prfs_poly[0])
print("recall", prfs_poly[1])
print("f-score", prfs_poly[2])

In [51]:
# rbf

vals = test_rbf_conf

plt.imshow(test_polyial_conf, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted')
plt.title('SVM RBF Confusion')

s = [['TP', 'FP'], ['FN', 'TN']]

for i in range(2):
    for j in range(2):
        plt.text(j, i, str(s[i][j]) + ' = ' + str(vals[i][j]))

print(test_linear_conf)
print("precision", prfs_rbf[0])
print("recall", prfs_rbf[1])
print("f-score", prfs_rbf[2])

## TESTING ON ORIGINAL unbalanced DATA

In [33]:
c_param = [0.01]
test_linear_conf, test_polyial_conf, test_rbf_conf, prfs_lin, prfs_poly, prfs_rbf, s1, s2,s3 = svm_(X_train1, y_train1, X_test1, y_test1, c_param)

In [43]:
print(s1)
print(s2)
print(s3)
plt.plot(c_param, s1, label = 'Linear Regression')
plt.plot(c_param, s2, label = 'Polynomial Regression')
plt.plot(c_param, s3, label = 'RBF Regression')
plt.xscale('log')
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title('Accuracy vs C for unbalanced')
plt.legend()
plt.show()

In [44]:
#     linear kernel

vals = test_linear_conf

plt.imshow(test_linear_conf, cmap='Greens', interpolation='nearest')
plt.colorbar()
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted')
plt.title('Linear Kernel Confusion Unbalanced')

s = [['TP', 'FP'], ['FN', 'TN']]

for i in range(2):
    for j in range(2):
        plt.text(j, i, str(s[i][j]) + ' = ' + str(vals[i][j]))


print(test_linear_conf)
print("precision", prfs_lin[0])
print("recall", prfs_lin[1])
print("f-score", prfs_lin[2])

In [45]:
# polyial kernel

vals = test_polyial_conf

plt.imshow(test_polyial_conf, cmap='Greens', interpolation='nearest')
plt.colorbar()
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted')
plt.title('SVM Poly Confusion Unbalanced')

s = [['TP', 'FP'], ['FN', 'TN']]

for i in range(2):
    for j in range(2):
        plt.text(j, i, str(s[i][j]) + ' = ' + str(vals[i][j]))

print(test_linear_conf)
print("precision", prfs_poly[0])
print("recall", prfs_poly[1])
print("f-score", prfs_poly[2])

In [46]:
# rbf

vals = test_rbf_conf

plt.imshow(test_polyial_conf, cmap='Greens', interpolation='nearest')
plt.colorbar()
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted')
plt.title('SVM RBF Confusion Unbalanced')

s = [['TP', 'FP'], ['FN', 'TN']]

for i in range(2):
    for j in range(2):
        plt.text(j, i, str(s[i][j]) + ' = ' + str(vals[i][j]))

print(test_linear_conf)
print("precision", prfs_rbf[0])
print("recall", prfs_rbf[1])
print("f-score", prfs_rbf[2])

## Neural NETWORKS


In [53]:
df1

In [86]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


scorel = []
epochl = []
itr = 0
def neural_network(X_train, y_train, X_test, y_test, act = "relu", r=None, inn=8, hidden=3):
    
    model = Sequential()
    model.add(Dense(inn, input_dim=30, activation=act, kernel_regularizer=r))
    model.add(Dense(hidden, activation=act))
    model.add(Dense(1, activation=act))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    score = model.evaluate(X_test, y_test, batch_size=32)
    
    return score[1]

In [108]:
    for i in [2, 10, 20, 50, 100, 500]:
            tot_acc.append(neural_network(X_train, y_train, X_test, y_test, inn=i, hidden=i))
    
# print("total acc: ", tot_acc)
# for i in tot_acc:
#     i/=2
# print("avg acc: ", tot_acc)

In [75]:
tot_acc_relu_L1 = 0
tot_acc_relu=0
tot_acc_tanh=0
tot_acc_tanh_L1=0
tot_acc_sigmoid=0
tot_acc_sigmoid_L1=0

for _ in range(10):
    tot_acc_relu += neural_network(X_train, y_train, X_test, y_test)
    tot_acc_tanh += neural_network(X_train, y_train, X_test, y_test, "tanh")
    tot_acc_sigmoid += neural_network(X_train, y_train, X_test, y_test, "tanh")
    
output = []
relu_avg = output.append(tot_acc_relu/10)
tanh_avg = output.append(tot_acc_tanh/10)
sigmoid_avg = output.append(tot_acc_sigmoid/10)

print("relu avg: ", output[0], "\n")
print("relu L1 avg: ", output[1], "\n")
print("tanh avg: ", output[2], "\n")
print("tanh L1 avg: ", output[3], "\n")
print("sigmoid avg: ", output[4], "\n")
print("sigmoid L1 avg: ", output[5], "\n")




# relu, tanh, sigmoid
# 


| Activation Function | Average Accuracy |
| --- | --- |
| ReLU | 0.7101538419723511 |
| Tanh | 0.8846153914928436 |
| Sigmoid | 0.8815384566783905 |

In [110]:
tot_acc_relu_L1 = 0
tot_acc_relu=0
tot_acc_tanh=0
tot_acc_tanh_L1=0
tot_acc_sigmoid=0
tot_acc_sigmoid_L1=0

tot_acc_relu += neural_network(X_train1, y_train1, X_test1, y_test1)
tot_acc_tanh += neural_network(X_train1, y_train1, X_test1, y_test1, "tanh")
tot_acc_sigmoid += neural_network(X_train1, y_train1, X_test1, y_test1, "tanh")
    
output = []
output.append(tot_acc_relu)
output.append(tot_acc_tanh)
output.append(tot_acc_sigmoid)

print("relu avg: ", output[0], "\n")
print("relu L1 avg: ", output[1], "\n")
print("tanh avg: ", output[2], "\n")
print("tanh L1 avg: ", output[3], "\n")
print("sigmoid avg: ", output[4], "\n")
print("sigmoid L1 avg: ", output[5], "\n")


| Activation Function | Average Accuracy |
| --- | --- |
| ReLU | 0.9994573593139648  |
| Tanh | 0.9994893074035645  |
| Sigmoid | 0.9994467496871948 |